# `tensorflow` version compatibility

following along [here](https://www.tensorflow.org/programmers_guide/version_compat)

In [1]:
import tensorflow as tf

import utils

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## semantic versioning 2.0

`MAJOR.MINOR.PATCH`

+ changes in `MAJOR` may break backwards compatability for everything (code, data, checkpoings, graphs)
+ changes in `MINOR` may break backwards compatability for anything *not in the public `api`*. **will not** break compatibility with public `api`
+ changein in `PATCH` should be backwards compatable

## what is covered

+ public `api`
    + all documented `python` functions and classes in `tensorflow`
        + **except**
            + `tf.contrib`
            + private functions of classes (`_***` names)
    + the `C` `api`
    + some protocol buffer files
        + `attr_value`
        + `config`
        + `event`
        + `graph`
        + `op_def`
        + `reader_base`
        + `summary`
        + `tensor`
        + `tensor_shape`
        + `types`

## what is not covered

+ `tf.contrib`
+ any language beyond `C` and `python` (e.g. `c++`, `java`, `go`)
+ implementation of composite ops (i.e. interface is BC, implementation is not)
+ exact floating point values
+ random numbers
+ bugs
+ error message text
+ error message type *if* it is not in the documentation

version skew in distributed environments is neither recommended nor supported

## compatibility of graphs and checkpoints

+ serialized graphs / checkpoints are done through the `GraphDef` protocol buffer.
    + they try hard not to fuck this up or change things too often
    + `GraphDef` pbs are versioned separately from `tensorflow`, so that is the fundamental topic of discussion
+ `tensorflow` versions each support a *range* of `GraphDef` versions
    + `PATCH` changes will not change this range
    + `MINOR` changes will only ever add to this range
    + `MAJOR` changes may possibly drop support for the oldest `GraphDef` versions

## graph and checkpoint compatibility when extending tensorflow

discussion of changes to the `GraphDef` version

### backward and partial forward compatibility

+ bc allows for loading of old checkpoints
+ (p)fc allows for producers of graphs to pass them to consumers at previous versions via `PATCH` releases in old versions

### independent data version schemes

graphs and checkpoints have different formats and therefore each has their own version number. check both of them (and `tensorflow`'s versions) out [on github](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/core/public/version.h)

### data, producers, and consumers

data objects are classified into two categories: *producer* or *consumer*. each type has its own version number *and also* the minimum version of the other with which it is compatible.

this is all defined in [the `VersionDef` protocol buffer](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/core/framework/versions.proto)

### add a new attribute with default to an existing op

to get forward compatability of your graph checkpoints, set `strip_default_attrs=True` when calling `add_meta_graph*` or `Estimator.export_savedmodel`

### evolving graphdef versions

advanced topic; let's skip for now and revisit if we ever create our own tf operation

# summary

the versioning scheme works basically how you expect. `MAJOR.MINOR.PATCH` with bc mostly protected in `MINOR.PATCH`. checkpoints and restores should work between `MAJOR` releases, with support being dropped at (and only at) `MAJOR` releases.